In [2]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/vol/tiny/home/koester/mambaforge/envs/snakemake/lib/python3.9/site-packages', '/vol/tiny/home/koester/scms/qc-benchmark-action/benchmarks/giab-na12878-exome/workflow/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\x9d\x04\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94\x8c\nreport.tsv\x94a}\x94(\x8c\x06_names\x94}\x94\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x10\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\x16)}\x94\x8c\x05_name\x94h\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8c\x0freport.plot.svg\x94a}\x94(h\x0c}\x94h\x0e]\x94(h\x10h\x11eh\x10h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94}\x94(h\x0c}\x94h\x0e]\x94(h\x10h\x11eh\x10h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94}\x94(h\x0c}\x94h\x0e]\x94(h\x10h\x11eh\x10h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\r/vol/tiny/tmp\x94e}\x94(h\x0c}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x0e]\x94(h\x10h\x11eh\x10h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bhVK\x01hXK\x01hZhSub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c\x1elogs/plot-precision-recall.log\x94a}\x94(h\x0c}\x94h\x0e]\x94(h\x10h\x11eh\x10h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x10sNt\x94bh\x11h\x14h\x16\x85\x94R\x94(h\x16)}\x94h\x1ah\x11sNt\x94bub\x8c\x06config\x94}\x94\x8c\x07results\x94\x8cn/vol/tiny/home/koester/scms/dna-seq-varlociraptor/.test/results/final-calls/NA12878.present.fdr-controlled.bcf\x94s\x8c\x04rule\x94\x8c\x15plot_precision_recall\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c`/vol/tiny/home/koester/scms/qc-benchmark-action/benchmarks/giab-na12878-exome/workflow/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/vol/tiny/home/koester/scms/qc-benchmark-action/benchmark-data-whole-genome');
######## snakemake preamble end #########


In [54]:
from itertools import product
import sys

import pandas as pd
import altair as alt
from altair import datum

sys.stderr = open(snakemake.log[0], "w")


report = pd.read_csv(snakemake.input[0], sep="\t")
columns = ["coverage", "Type", "precision", "recall", "correct genotype"]
report = report.loc[report["Filter"] == "PASS", ]
report["Type"] = report["Type"].apply(lambda t: "indels" if t == "INDEL" else "snvs")
report["coverage"] = report["coverage"].apply(lambda cov: snakemake.params.cov_labels[cov])

report["tp_no_gt"] = report["QUERY.TOTAL"] - report["QUERY.FP"] + report["FP.gt"]
report["t"] = report["tp_no_gt"] + report["TRUTH.FN"]
report["precision"] = report["tp_no_gt"] / report["QUERY.TOTAL"]
report["recall"] = report["tp_no_gt"] / (report["t"])
report["correct genotype"] = "no"
report["precision_numbers"] = report.apply(lambda row: f"{row['tp_no_gt']}/{row['QUERY.TOTAL']}", axis=1)
report["recall_numbers"] = report.apply(lambda row: f"{row['tp_no_gt']}/{row['t']}", axis=1)

report_gt = report[["coverage", "Type", "METRIC.Precision", "METRIC.Recall"]]
report_gt["correct genotype"] = "yes"
report_gt.columns = columns

report_final = pd.concat([report[columns], report_gt])
report_final.columns = report_final.columns.str.lower()

report_final = report_final.set_index(["coverage", "type", "correct genotype"]).stack().reset_index()
report_final.columns = ["coverage", "type", "correct genotype", "metric", "value"]

coverages = report_final["coverage"].unique()
vartypes = report_final["type"].unique()

report_final.set_index(["coverage", "type", "metric"], inplace=True)
report_final["numbers"] = None
for cov, vartype in product(coverages, vartypes):
    report_ = report.loc[(report["coverage"] == cov) & (report["Type"] == vartype)].iloc[0]
    report_final.loc[(cov, vartype, "precision"), "numbers"] = f"{report_.at['tp_no_gt']}/{report_.at['QUERY.TOTAL']}"
    report_final.loc[(cov, vartype, "recall"), "numbers"] = f"{report_.at['tp_no_gt']}/{report_.at['t']}"
report_final.reset_index(inplace=True)

def plot(correct_genotype, print_numbers=True):
    chart = alt.Chart().mark_bar().encode(
        alt.X("value", title=None),
        alt.Y("coverage", sort=["high", "medium", "low"]),
        color=alt.Color("correct genotype", type="ordinal", sort="descending"),
    ).transform_filter(
        datum["correct genotype"] == correct_genotype
    )

    if print_numbers:
        return chart + alt.Chart().mark_text(
            color="black",
            align="right",
            baseline="middle",
            dx=2,
        ).encode(
            alt.X("value", title=None),
            alt.Y("coverage", sort=["high", "medium", "low"]),
            text="numbers"
        )
    else:
        return chart

chart = alt.layer(plot("no"), plot("yes"), data=report_final).facet(
    row=alt.Row("metric", title=None),
    column=alt.Column("type", title=None, sort=["snvs", "indels"]),
)

chart


/vol/tiny/tmp/ipykernel_23506/677199457.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_gt["correct genotype"] = "yes"


alt.FacetChart(...)

In [55]:
chart.save(snakemake.output[0])